# Numerical Optimization

This notebook introduces numerical optimization methods in machine learning that minimize loss functions efficiently and stably. The focus is on understanding the intuition, math, and convergence behavior of common optimizers:
- Gradient descent and learning rate schedules
- Stochastic gradient descent (SGD) and mini-batching
- Momentum and Nesterov accelerated gradients
- Adaptive methods (AdaGrad, RMSProp, Adam)
- Convergence analysis and visulization of loss surfaces for each algorithm